# Introduction

Starting with OGR 1.10, the SQLite SQL engine can be used to run SQL queries on any OGR datasource if using the SQLite SQL dialect.

This notebook documents how you might go about doing so, and follows the cookbook in https://www.gaia-gis.it/gaia-sins/spatialite-cookbook/.

In [1]:
import ArchGDAL; const AG = ArchGDAL

ArchGDAL

## SQLite / Spatialite RDBMS
(taken from http://gdal.org/drv_sqlite.html)

OGR optionally supports spatial and non-spatial tables stored in SQLite 3.x database files. SQLite is a "light weight" single file based RDBMS engine with fairly complete SQL semantics and respectable performance.

The driver can handle "regular" SQLite databases, as well as Spatialite databases (spatial enabled SQLite databases). The type of an existing database can be checked from the SQLITE debug info value "OGR style SQLite DB found/ SpatiaLite DB found/SpatiaLite v4 DB found" obtained by running "ogrinfo db.sqlite --debug on"

By default, SQL statements are passed directly to the SQLite database engine. It's also possible to request the driver to handle SQL commands with OGR SQL engine, by passing "OGRSQL" string to the ExecuteSQL() method, as name of the SQL dialect.

### "Regular" SQLite databases

The driver looks for a geometry_columns table laid out as defined loosely according to OGC Simple Features standards, particularly as defined in [FDO RFC 16](https://trac.osgeo.org/fdo/wiki/FDORfc16). If found it is used to map tables to layers.

If `geometry_columns` is not found, each table is treated as a layer. Layers with a `WKT_GEOMETRY` field will be treated as spatial tables, and the `WKT_GEOMETRY` column will be read as Well Known Text geometry.

If `geometry_columns` is found, it will be used to lookup spatial reference systems in the `spatial_ref_sys` table.

While the SQLite driver supports reading spatial data from records, there is no support for spatial indexing, so spatial queries will tend to be slow (use Spatialite for that). Attributes queries may be fast, especially if indexes are built for appropriate attribute columns using the `"CREATE INDEX ON ( )"` SQL command.

Starting with GDAL 2.0, the driver also supports reading and writing the following non-linear geometry types: `CIRCULARSTRING`, `COMPOUNDCURVE`, `CURVEPOLYGON`, `MULTICURVE` and `MULTISURFACE`. Note: this is not true for Spatialite databases, since those geometry types are not supported by current Spatialite versions.

### Using the SpatiaLite library (Spatial extension for SQLite)
(Starting with GDAL 1.7.0)

The SQLite driver can read and write SpatiaLite databases. Creating or updating a spatialite database requires explicit linking against SpatiaLite library (version >= 2.3.1). Explicit linking against SpatiaLite library also provides access to functions provided by this library, such as spatial indexes, spatial functions, etc...

### Other Remarks
The SQLite database is essentially typeless, but the SQLite driver will attempt to classify attributes field as text, integer or floating point based on the contents of the first record in a table. None of the list attribute field types existing in SQLite. Starting with OGR 1.10, datetime field types are also handled.

SQLite databases often due not work well over NFS, or some other networked file system protocols due to the poor support for locking. It is safest to operate only on SQLite files on a physical disk of the local system.

# Preliminaries

## Datasets

The first dataset we'll use is the Italian National Census 2001, kindly released by ISTAT	(the Italian Census Bureau). In particular, we'll be working with the following files (from `test/spatialite-cookbook`):
- Censimento 2001 - Regioni (Regions):
http://www3.istat.it/ambiente/cartografia/regioni2001.zip
- Censimento 2001 - Province (Counties):
http://www3.istat.it/ambiente/cartografia/province2001.zip
- Censimento 2001 - Comuni (Local Councils):
http://www3.istat.it/ambiente/cartografia/comuni2001.zip

The second required dataset is GeoNames, a worldwide collection of Populated Places. There are several flavors of this dataset. We'll use cities-1000 (any populated place into the word counting more than 1,000 peoples): http://download.geonames.org/export/dump/cities1000.zip

The files can be found in:

In [2]:
;ls ../test/spatialite-cookbook/

cities1000.txt
cities1000.zip
comuni2001
comuni2001.zip
cookbook-data.zip
province2001
regioni2001


## VSI Virtual File System API support

The driver supports reading and writing to files managed by VSI Virtual File System API, which include "regular" files, as well as files in the /vsimem/ (read-write), /vsizip/ (read-only), /vsigzip/ (read-only), /vsicurl/ (read-only) domains.

read https://trac.osgeo.org/gdal/wiki/UserDocs/ReadInZip for more

In [3]:
AG.registerdrivers() do
    filepath = "../test/spatialite-cookbook/cookbook-data.zip"
    AG.read("/vsizip/$filepath/comuni2001/com2001_s.shp") do dataset
        print(dataset)
    end
end;

GDAL Dataset (Driver: ESRI Shapefile/ESRI Shapefile)
Files: /vsizip/../test/spatialite-cookbook/cookbook-data.zip/comuni2001/com2001_s.shp
       /vsizip/../test/spatialite-cookbook/cookbook-data.zip/comuni2001/com2001_s.shx
       /vsizip/../test/spatialite-cookbook/cookbook-data.zip/comuni2001/com2001_s.dbf
       /vsizip/../test/spatialite-cookbook/cookbook-data.zip/comuni2001/com2001_s.prj
       /vsizip/../test/spatialite-cookbook/cookbook-data.zip/comuni2001/com2001_s.sbn
       /vsizip/../test/spatialite-cookbook/cookbook-data.zip/comuni2001/com2001_s.sbx

Number of feature layers: 1
  Layer 0: com2001_s (wkbPolygon), nfeatures = 8101


## The VirtualOGR SQLite extension

Starting with OGR 1.10, the GDAL/OGR library can be loaded as a SQLite extension. This is automatically done when using SQLite through GDAL/OGR. After the extension is loaded, a virtual table, corresponding to a OGR layer, can be created with any one of the following SQL statements:

    CREATE VIRTUAL TABLE table_name USING VirtualOGR(datasource_name);
    CREATE VIRTUAL TABLE table_name USING VirtualOGR(datasource_name, update_mode);
    CREATE VIRTUAL TABLE table_name USING VirtualOGR(datasource_name, update_mode, layer_name);
    CREATE VIRTUAL TABLE table_name USING VirtualOGR(datasource_name, update_mode, layer_name, expose_ogr_style);

where

- `datasource_name` is the connection string to any OGR datasource.
- `update_mode` = `0` for read-only mode (default value) or `1` for update mode.
- `layer_name` = the name of a layer of the opened datasource.
- `expose_ogr_style` = `0` to prevent the OGR_STYLE special from being displayed (default value) or `1` to expose it.

**Note**: layer_name does not need to be specified if the datasource has only one single layer.

In [4]:
AG.registerdrivers() do
    AG.create("","SQLite") do ds
        filename = "../test/spatialite-cookbook/cookbook-data.zip/comuni2001/com2001_s.shp"
        AG.unsafe_executesql(ds, "CREATE VIRTUAL TABLE com2001 USING VirtualOGR('/vsizip/$filename')")
        print(ds)
    end
end

GDAL Dataset (Driver: SQLite/SQLite / Spatialite)

Number of feature layers: 1
  Layer 0: com2001 (wkbPolygon), nfeatures = 8101


Alternatively, you can use the `ogr_datasource_load_layers(datasource_name[, update_mode[, prefix]])` function to automatically load all the layers of a datasource. For example:

    SELECT ogr_datasource_load_layers('filepath')
    SELECT * FROM sqlite_master

In [5]:
AG.registerdrivers() do
    AG.create("","SQLite") do ds
        filename = "../test/spatialite-cookbook/cookbook-data.zip/comuni2001/com2001_s.shp"
        AG.unsafe_executesql(ds, "SELECT ogr_datasource_load_layers('/vsizip/$filename')")
        AG.executesql(ds, "SELECT * FROM sqlite_master") do result
            print(result)
        end
    end
end

Layer: SELECT, nfeatures = 4
     Field 0 (type): [OFTString], table, table, index, table
     Field 1 (name): [OFTString], geometry_columns, spatial_ref_sys, ...
     Field 2 (tbl_name): [OFTString], geometry_columns, spatial_ref_sys, ...
     Field 3 (rootpage): [OFTInteger], 2, 3, 4, 0
     Field 4 (sql): [OFTString], CREATE TABLE geometr..., ...


false

## Opening with 'VirtualShape:'

It is possible to open on-the-fly a shapefile as a VirtualShape with Spatialite. The syntax to use for the datasource is "VirtualShape:/path/to/shapefile.shp" (the shapefile must be a "real" file).

In [6]:
AG.registerdrivers() do
    AG.read("VirtualShape:../test/spatialite-cookbook/comuni2001/com2001_s.shp") do dataset
        print(dataset)
    end
end;

GDAL Dataset (Driver: SQLite/SQLite / Spatialite)

Number of feature layers: 1
  Layer 0: com2001_s (wkbMultiPolygon), nfeatures = 8101


read http://gdal.org/drv_sqlite.html for more.

# Spatialite Cookbook
starts here. If this is your first introduction to working with SQL, I highly recommend you read the whole cookbook at https://www.gaia-gis.it/gaia-sins/spatialite-cookbook/ as well.

In [7]:
function inspect(sqlcommand)
    AG.registerdrivers() do
        AG.create("","SQLite", options=["SPATIALITE=YES"]) do ds
            filepath = "/vsizip/../test/spatialite-cookbook/cookbook-data.zip"
            for (layername, filename) in (
                    ("com2001_s",  "/comuni2001/com2001_s.shp"),
                    ("prov2001_s", "/province2001/prov2001_s.shp"),
                    ("reg2001_s",  "/regioni2001/reg2001_s.shp"))
                sql = "CREATE VIRTUAL TABLE $layername USING VirtualOGR('$filepath$filename')"
                AG.unsafe_executesql(ds, sql)
            end
            AG.executesql(ds, sqlcommand) do result
                print(result)
            end
        end
    end
end

inspect (generic function with 1 method)

In [8]:
inspect("""
SELECT * 
FROM reg2001_s
""")

Layer: SELECT, nfeatures = 20
  Geometry 0 (GEOMETRY): [wkbUnknown], POLYGON ((457832.312...), ...
     Field 0 (COD_REG): [OFTInteger], 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, ...
     Field 1 (REGIONE): [OFTString], PIEMONTE, VALLE D'AOSTA, LOMBARDIA, ...
     Field 2 (POP2001): [OFTInteger], 4214677, 119548, 9032554, 940016, ...


false

In [9]:
inspect("""
SELECT pop2001, regione
FROM   reg2001_s
""")

Layer: SELECT, nfeatures = 20
     Field 0 (POP2001): [OFTInteger], 4214677, 119548, 9032554, 940016, ...
     Field 1 (REGIONE): [OFTString], PIEMONTE, VALLE D'AOSTA, LOMBARDIA, ...


false

In [10]:
inspect("""
SELECT Cod_rEg AS code, REGIONE AS name,
  pop2001 AS "population (2001)"
FROM reg2001_s
""")

Layer: SELECT, nfeatures = 20
     Field 0 (code): [OFTInteger], 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, ...
     Field 1 (name): [OFTString], PIEMONTE, VALLE D'AOSTA, LOMBARDIA, ...
     Field 2 (population (2001)): [OFTInteger], 4214677, 119548, 9032554, ...


false

In [11]:
inspect("""
SELECT COD_REG, REGIONE, POP2001
FROM reg2001_s
ORDER BY regione
""")

Layer: SELECT, nfeatures = 20
     Field 0 (COD_REG): [OFTInteger], 13, 17, 18, 15, 8, 6, 12, 7, 3, 11, ...
     Field 1 (REGIONE): [OFTString], ABRUZZO, BASILICATA, CALABRIA, ...
     Field 2 (POP2001): [OFTInteger], 1262392, 597768, 2011466, 5701931, ...


false

In [12]:
inspect("""
SELECT COD_REG, REGIONE, POP2001
FROM reg2001_s
ORDER BY POP2001 DESC
""")

Layer: SELECT, nfeatures = 20
     Field 0 (COD_REG): [OFTInteger], 3, 15, 12, 19, 5, 1, 16, 8, 9, 18, 20, ...
     Field 1 (REGIONE): [OFTString], LOMBARDIA, CAMPANIA, LAZIO, SICILIA, ...
     Field 2 (POP2001): [OFTInteger], 9032554, 5701931, 5112413, 4968991, ...


false

In [13]:
inspect("""
SELECT COD_PRO, PROVINCIA, SIGLA
FROM prov2001_s
WHERE COD_REG = 9
""")

Layer: SELECT, nfeatures = 10
     Field 0 (COD_PRO): [OFTInteger], 45, 46, 47, 48, 49, 50, 51, 52, 53, ...
     Field 1 (PROVINCIA): [OFTString], MASSA CARRARA, LUCCA, PISTOIA, ...
     Field 2 (SIGLA): [OFTString], MS, LU, PT, FI, LI, PI, AR, SI, GR, PO


false

In [14]:
inspect("""
SELECT PRO_COM, NOME_COM, POP2001
FROM com2001_s
WHERE COD_PRO = 48
""")

Layer: SELECT, nfeatures = 44
     Field 0 (PRO_COM): [OFTInteger], 48001, 48002, 48003, 48004, 48005, ...
     Field 1 (NOME_COM): [OFTString], BAGNO A RIPOLI, BARBERINO DI MUGELLO, ...
     Field 2 (POP2001): [OFTInteger], 25232, 9531, 3871, 15825, 15042, ...


false

In [15]:
inspect("""
SELECT PRO_COM, NOME_COM, POP2001
FROM com2001_s
WHERE COD_REG = 9 AND POP2001 > 50000
ORDER BY POP2001 DESC
""")

Layer: SELECT, nfeatures = 13
     Field 0 (PRO_COM): [OFTInteger], 48017, 100005, 49009, 51002, 50026, ...
     Field 1 (NOME_COM): [OFTString], FIRENZE, PRATO, LIVORNO, AREZZO, PISA, ...
     Field 2 (POP2001): [OFTInteger], 356118, 172499, 156274, 91589, 89694, ...


false

In [16]:
inspect("""
SELECT PRO_COM, NOME_COM, POP2001
FROM com2001_s
WHERE nome_com = 'ROMA'
""")

Layer: SELECT, nfeatures = 1
     Field 0 (PRO_COM): [OFTInteger], 58091
     Field 1 (NOME_COM): [OFTString], ROMA
     Field 2 (POP2001): [OFTInteger], 2546804


false

In [17]:
inspect("""
SELECT PRO_COM, NOME_COM, POP2001
FROM com2001_s
WHERE nome_com = 'L''AQUILA'
""")

Layer: SELECT, nfeatures = 1
     Field 0 (PRO_COM): [OFTInteger], 66049
     Field 1 (NOME_COM): [OFTString], L'AQUILA
     Field 2 (POP2001): [OFTInteger], 68503


false

In [18]:
inspect("""
SELECT PRO_COM, NOME_COM, POP2001
FROM com2001_s
WHERE nome_com LIKE 'roma'
""")

Layer: SELECT, nfeatures = 1
     Field 0 (PRO_COM): [OFTInteger], 58091
     Field 1 (NOME_COM): [OFTString], ROMA
     Field 2 (POP2001): [OFTInteger], 2546804


false

In [19]:
inspect("""
SELECT PRO_COM, NOME_COM, POP2001
FROM com2001_s
WHERE nome_com LIKE '%maria%'
""")

Layer: SELECT, nfeatures = 27
     Field 0 (PRO_COM): [OFTInteger], 13143, 13210, 14067, 18143, 20032, ...
     Field 1 (NOME_COM): [OFTString], MARIANO COMENSE, ...
     Field 2 (POP2001): [OFTInteger], 20282, 1088, 892, 2584, 594, 13685, ...


false

In [20]:
inspect("""
SELECT PRO_COM, NOME_COM, POP2001
FROM com2001_s
WHERE nome_com IN ('ROMA', 'MILANO', 'NAPOLI')
""")

Layer: SELECT, nfeatures = 3
     Field 0 (PRO_COM): [OFTInteger], 15146, 58091, 63049
     Field 1 (NOME_COM): [OFTString], MILANO, ROMA, NAPOLI
     Field 2 (POP2001): [OFTInteger], 1256211, 2546804, 1004500


false

In [21]:
inspect("""
SELECT PRO_COM, NOME_COM, POP2001
FROM com2001_s
WHERE POP2001 BETWEEN 1990 AND 2010
""")

Layer: SELECT, nfeatures = 26
     Field 0 (PRO_COM): [OFTInteger], 1027, 1222, 5012, 5087, 8045, 9068, ...
     Field 1 (NOME_COM): [OFTString], BOLLENGO, ROLETTO, BUTTIGLIERA D'ASTI, ...
     Field 2 (POP2001): [OFTInteger], 1997, 1994, 1996, 1992, 2002, 1991, ...


false

In [22]:
inspect("""
SELECT PROVINCIA, SIGLA, POP2001
FROM prov2001_s
WHERE COD_REG IN (9, 10, 11, 12)
  AND SIGLA NOT IN ('LI', 'PI')
  AND (POP2001 BETWEEN 300000 AND 500000
    OR POP2001 > 750000)
""")

Layer: SELECT, nfeatures = 10
     Field 0 (PROVINCIA): [OFTString], LUCCA, FIRENZE, AREZZO, ...
     Field 1 (SIGLA): [OFTString], LU, FI, AR, PS, AN, MC, AP, RM, LT, FR
     Field 2 (POP2001): [OFTInteger], 372244, 933860, 323288, 351214, ...


false

In [23]:
inspect("""
SELECT *
FROM com2001_s
LIMIT 10
""")

Layer: SELECT, nfeatures = 10
  Geometry 0 (GEOMETRY): [wkbUnknown], POLYGON ((405246.812...), ...
     Field 0 (PRO_COM): [OFTInteger], 1001, 1002, 1003, 1004, 1005, 1006, ...
     Field 1 (COD_REG): [OFTInteger], 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
     Field 2 (COD_PRO): [OFTInteger], 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
     Field 3 (NOME_COM): [OFTString], AGLIE', AIRASCA, ALA DI STURA, ...
     Field 4 (POP2001): [OFTInteger], 2574, 3554, 479, 1696, 616, 5658, 300, ...


false

In [24]:
inspect("""
SELECT *
FROM com2001_s
LIMIT 10 OFFSET 1000
""")

Layer: SELECT, nfeatures = 10
  Geometry 0 (GEOMETRY): [wkbUnknown], MULTIPOLYGON (((4671...), ...
     Field 0 (PRO_COM): [OFTInteger], 6144, 6145, 6146, 6147, 6148, 6149, ...
     Field 1 (COD_REG): [OFTInteger], 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
     Field 2 (COD_PRO): [OFTInteger], 6, 6, 6, 6, 6, 6, 6, 6, 6, 6
     Field 3 (NOME_COM): [OFTString], RIVALTA BORMIDA, RIVARONE, ...
     Field 4 (POP2001): [OFTInteger], 1443, 372, 167, 1346, 220, 1650, 475, ...


false

### Understanding Aggregate Functions

In [25]:
inspect("""
SELECT Min(POP2001), Max(POP2001),
  Avg(POP2001), Sum(POP2001), Count(*)
FROM com2001_s
""")

Layer: SELECT, nfeatures = 1
     Field 0 (Min(POP2001)): [OFTInteger], 33
     Field 1 (Max(POP2001)): [OFTInteger], 2546804
     Field 2 (Avg(POP2001)): [OFTReal], 7035.6430070361685
     Field 3 (Sum(POP2001)): [OFTInteger], 56995744
     Field 4 (Count(*)): [OFTInteger], 8101


false

In [26]:
inspect("""
SELECT COD_PRO, Min(POP2001), Max(POP2001),
  Avg(POP2001), Sum(POP2001), Count(*)
FROM com2001_s
GROUP BY COD_PRO
""")

Layer: SELECT, nfeatures = 103
     Field 0 (COD_PRO): [OFTInteger], 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, ...
     Field 1 (Min(POP2001)): [OFTInteger], 46, 49, 260, 56, 104, 95, 91, ...
     Field 2 (Max(POP2001)): [OFTInteger], 865263, 45132, 100910, 52334, ...
     Field 3 (Avg(POP2001)): [OFTReal], 6874.980952380953, ...
     Field 4 (Sum(POP2001)): [OFTInteger], 2165619, 176829, 343040, 556330, ...
...
 Number of Fields: 6

In [27]:
inspect("""
SELECT COD_REG, Min(POP2001), Max(POP2001),
  Avg(POP2001), Sum(POP2001), Count(*)
FROM com2001_s
GROUP BY COD_REG
""")

Layer: SELECT, nfeatures = 20
     Field 0 (COD_REG): [OFTInteger], 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, ...
     Field 1 (Min(POP2001)): [OFTInteger], 46, 91, 33, 105, 128, 195, 95, ...
     Field 2 (Max(POP2001)): [OFTInteger], 865263, 34062, 1256211, 104946, ...
     Field 3 (Avg(POP2001)): [OFTReal], 3494.757048092869, ...
     Field 4 (Sum(POP2001)): [OFTInteger], 4214677, 119548, 9032554, 940016, ...
...
 Number of Fields: 6

In [28]:
inspect("""
SELECT DISTINCT COD_REG, COD_PRO
FROM com2001_s
ORDER BY COD_REG, COD_PRO
""")

Layer: SELECT, nfeatures = 103
     Field 0 (COD_REG): [OFTInteger], 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3, 3, 3, ...
     Field 1 (COD_PRO): [OFTInteger], 1, 2, 3, 4, 5, 6, 96, 103, 7, 12, 13, ...


false

### Your First SQL Spatial Queries

In [29]:
inspect("""
SELECT COD_REG, REGIONE, ST_Area(Geometry)
FROM reg2001_s
""")

Layer: SELECT, nfeatures = 20
     Field 0 (COD_REG): [OFTInteger], 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, ...
     Field 1 (REGIONE): [OFTString], PIEMONTE, VALLE D'AOSTA, LOMBARDIA, ...
     Field 2 (ST_Area(Geometry)): [OFTReal], 2.5395423847624672e10, ...


false

In [30]:
inspect("""
SELECT COD_REG AS code,
  REGIONE AS name,
  ST_Area(Geometry) / 1000000.0 AS "Surface (sq.Km)"
FROM reg2001_s
ORDER BY 3 DESC
""")

Layer: SELECT, nfeatures = 20
     Field 0 (code): [OFTInteger], 19, 1, 20, 3, 9, 8, 16, 5, 12, 18, 15, 4, ...
     Field 1 (name): [OFTString], SICILIA, PIEMONTE, SARDEGNA, LOMBARDIA, ...
     Field 2 (Surface (sq.Km)): [OFTReal], 25735.673661027224, ...


false

In [31]:
inspect("""
SELECT COD_REG AS code,
  REGIONE AS name,
  ST_Area(Geometry) / 1000000.0 AS "Surface (sq.Km)",
  POP2001 / (ST_Area(Geometry) / 1000000.0)
    AS "Density: Peoples / sq.Km"
FROM reg2001_s
ORDER BY 4 DESC
""")

Layer: SELECT, nfeatures = 20
     Field 0 (code): [OFTInteger], 15, 3, 12, 7, 5, 16, 19, 8, 1, 9, 11, 6, ...
     Field 1 (name): [OFTString], CAMPANIA, LOMBARDIA, LAZIO, LIGURIA, ...
     Field 2 (Surface (sq.Km)): [OFTReal], 13666.322145371321, ...
     Field 3 (Density: Peoples / sq.Km): [OFTReal], 417.22498118714407, ...


false

### More about Spatial SQL: WKT and WKB

In [32]:
inspect("""
SELECT Hex(ST_GeomFromText('POINT(1.2345 2.3456)'))
""")

Layer: SELECT, nfeatures = 1
     Field 0 (Hex(ST_GeomFromText('POINT(1.2345 2.3456)'))): [OFTString], ...


false

In [33]:
inspect("""
SELECT ST_AsText(x'0001FFFFFFFF8D976E1283C0F33F16FBCBEEC9C302408D976E1283C0F33F16FBCBEEC9C302407C010000008D976E1283C0F33F16FBCBEEC9C30240FE')
""")

Layer: SELECT, nfeatures = 1
     Field 0 (ST_AsText(x'0001FFFFFFFF8D976E1283C0F33F16FBCBEEC9C302408D976E1283C0F33F16FBCBEEC9C302407C010000008D976E1283C0F33F16FBCBEEC9C30240FE')): [OFTString][1:70]...


false

In [34]:
inspect("""
SELECT Hex(ST_AsBinary(x'0001FFFFFFFF8D976E1283C0F33F16FBCBEEC9C302408D976E1283C0F33F16FBCBEEC9C302407C010000008D976E1283C0F33F16FBCBEEC9C30240FE'))
""")

Layer: SELECT, nfeatures = 1
     Field 0 (Hex(ST_AsBinary(x'0001FFFFFFFF8D976E1283C0F33F16FBCBEEC9C302408D976E1283C0F33F16FBCBEEC9C302407C010000008D976E1283C0F33F16FBCBEEC9C30240FE'))): [OFTString][1:70]...


false

In [35]:
inspect("""
SELECT Hex(ST_AsBinary(ST_GeomFromText('POINT(1.2345 2.3456)')))
""")

Layer: SELECT, nfeatures = 1
     Field 0 (Hex(ST_AsBinary(ST_GeomFromText('POINT(1.2345 2.3456)')))): [OFTString][1:70]...


false

In [36]:
inspect("""
SELECT ST_AsText(ST_GeomFromWKB(x'01010000008D976E1283C0F33F16FBCBEEC9C30240'))
""")

Layer: SELECT, nfeatures = 1
     Field 0 (ST_AsText(ST_GeomFromWKB(x'01010000008D976E1283C0F33F16FBCBEEC9C30240'))): [OFTString][1:70]...


false

In [37]:
inspect("""
SELECT ST_GeometryType(ST_GeomFromText('POINT(1.2345 2.3456)'))
""")

Layer: SELECT, nfeatures = 1
     Field 0 (ST_GeometryType(ST_GeomFromText('POINT(1.2345 2.3456)'))): [OFTString][1:70]...


false

In [38]:
inspect("""
SELECT ST_GeometryType(ST_GeomFromText('POINTZ(1.2345 2.3456 10)'))
""")

Layer: SELECT, nfeatures = 1
     Field 0 (ST_GeometryType(ST_GeomFromText('POINTZ(1.2345 2.3456 10)'))): [OFTString][1:70]...


false

In [39]:
inspect("""
SELECT ST_GeometryType(ST_GeomFromText('POINT ZM(1.2345 2.3456 10 20)'))
""")

Layer: SELECT, nfeatures = 1
     Field 0 (ST_GeometryType(ST_GeomFromText('POINT ZM(1.2345 2.3456 10 20)'))): [OFTString][1:70]...


false

In [40]:
inspect("""
SELECT ST_Srid(ST_GeomFromText('POINT(1.2345 2.3456)'))
""")

Layer: SELECT, nfeatures = 1
     Field 0 (ST_Srid(ST_GeomFromText('POINT(1.2345 2.3456)'))): [OFTInteger][1:70]...


false

In [41]:
inspect("""
SELECT ST_Srid(ST_GeomFromText('POINT(1.2345 2.3456)', 4326))
""")

Layer: SELECT, nfeatures = 1
     Field 0 (ST_Srid(ST_GeomFromText('POINT(1.2345 2.3456)', 4326))): [OFTInteger][1:70]...


false

### Common PitFalls

In [42]:
inspect("""
SELECT ST_GeometryType(ST_GeomFromText('MULTIPOINT(1.2345 2.3456)'))
""")

Layer: SELECT, nfeatures = 1
     Field 0 (ST_GeometryType(ST_GeomFromText('MULTIPOINT(1.2345 2.3456)'))): [OFTString][1:70]...


false

In [43]:
inspect("""
SELECT ST_AsText(CastToMultiLineString(ST_GeomFromText('LINESTRING(1.2345 2.3456, 12.3456 23.4567)')))
""")

Layer: SELECT, nfeatures = 1
     Field 0 (ST_AsText(CastToMultiLineString(ST_GeomFromText('LINESTRING(1.2345 2.3456, 12.3456 23.4567)')))): [OFTString][1:70]...


false

In [44]:
inspect("""
SELECT ST_AsText(CastToXYZM(ST_GeomFromText('POINT(1.2345 2.3456)')))
""")

Layer: SELECT, nfeatures = 1
     Field 0 (ST_AsText(CastToXYZM(ST_GeomFromText('POINT(1.2345 2.3456)')))): [OFTString][1:70]...


false

In [45]:
inspect("""
SELECT ST_AsText(CastToXY(ST_GeomFromText('POINT ZM(1.2345 2.3456 10 20)')))
""")

Layer: SELECT, nfeatures = 1
     Field 0 (ST_AsText(CastToXY(ST_GeomFromText('POINT ZM(1.2345 2.3456 10 20)')))): [OFTString][1:70]...


false

In [46]:
inspect("""
SELECT Hex(ST_GeomFromText('POINT(1.2345 2.3456)'))
""")

Layer: SELECT, nfeatures = 1
     Field 0 (Hex(ST_GeomFromText('POINT(1.2345 2.3456)'))): [OFTString], ...


false

### Spatial MetaData Tables

In [47]:
inspect("""
SELECT InitSpatialMetaData()
""")

Layer: SELECT, nfeatures = -1


InitSpatiaMetaData() error:"table spatial_ref_sys already exists"


LoadError: LoadError: GDALError
	GDAL returned nothing
while loading In[47], in expression starting on line 1

In [48]:
inspect("""
SELECT * FROM spatial_ref_sys
""")

Layer: SELECT, nfeatures = 4924
     Field 0 (srid): [OFTInteger], -1, 0, 2000, 2001, 2002, 2003, 2004, ...
     Field 1 (auth_name): [OFTString], NONE, NONE, epsg, epsg, epsg, epsg, ...
     Field 2 (auth_srid): [OFTInteger], -1, 0, 2000, 2001, 2002, 2003, 2004, ...
     Field 3 (ref_sys_name): [OFTString], Undefined - Cartesian, ...
     Field 4 (proj4text): [OFTString], , , +proj=tmerc +lat_0=0..., ...
...
 Number of Fields: 6

In [49]:
inspect("""
SELECT * FROM geometry_columns
""")

Layer: SELECT, nfeatures = 0
     Field 0 (f_table_name): [OFTString]
     Field 1 (f_geometry_column): [OFTString]
     Field 2 (geometry_type): [OFTInteger]
     Field 3 (coord_dimension): [OFTInteger]
     Field 4 (srid): [OFTInteger]
...
 Number of Fields: 6